In [6]:
!pip install ultralytics
!pip install roboflow
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 110.9 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 14.6 MB/s eta 0:00:00


In [7]:
from roboflow import Roboflow
import optuna
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [15]:
rf = Roboflow(api_key=MY_SECRET_KEY)
project = rf.workspace("-jwzpw").project("continuous_fire")
version = project.version(6)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to continuous_fire-6 in yolov8:: 100%|██████████| 5030/5030 [00:00<00:00, 12589.14it/s]


In [17]:
import yaml
#update path for dataset
yaml_path = "/content/continuous_fire-6/data.yaml"
root = "/content/continuous_fire-6"

with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

data.update({
    "train": f"{root}/train/images",
    "val":   f"{root}/valid/images",
    "test":  f"{root}/test/images"
})

with open(yaml_path, "w") as f:
    yaml.safe_dump(data, f)

In [18]:
import os
import csv

#Paths
DATA_YAML = "/content/continuous_fire-6/data.yaml"
RESULTS_DIR = "./results"
os.makedirs(RESULTS_DIR, exist_ok=True)
CSV_FILE = os.path.join(RESULTS_DIR, "optuna_trials.csv")

#CSV to store trial results
with open(CSV_FILE, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([
        'trial_number', 'lr', 'momentum', 'weight_decay',
        'augment', 'mosaic', 'mixup', 'hsv_h', 'hsv_s', 'hsv_v',
        'map50'
    ])


In [16]:
from ultralytics import YOLO
import optuna

def objective(trial):
    # Suggest hyperparameters
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    momentum = trial.suggest_uniform('momentum', 0.6, 0.99)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
    augment = trial.suggest_categorical('augment', [True, False])
    mosaic = trial.suggest_uniform('mosaic', 0.0, 1.0)
    mixup = trial.suggest_uniform('mixup', 0.0, 1.0)
    hsv_h = trial.suggest_uniform('hsv_h', 0.0, 0.5)
    hsv_s = trial.suggest_uniform('hsv_s', 0.0, 0.7)
    hsv_v = trial.suggest_uniform('hsv_v', 0.0, 0.5)

    # Initialize YOLOv8
    model = YOLO("yolov8n.pt")  # small YOLOv8 for quick trials

    # Train briefly for hyperparameter evaluation
    model.train(
        data=DATA_YAML,
        epochs=10,
        imgsz=640,
        batch=16,
        lr0=lr,
        momentum=momentum,
        weight_decay=weight_decay,
        augment=augment,
        mosaic=mosaic,
        mixup=mixup,
        hsv_h=hsv_h,
        hsv_s=hsv_s,
        hsv_v=hsv_v,
        project=RESULTS_DIR,
        name=f"optuna_trial_{trial.number}",
        exist_ok=True,
        save=False
    )

    #validate
    metrics = model.val(data=DATA_YAML, save_json=False)
    map50 = metrics.box.map50

    #log trial results to CSV
    with open(CSV_FILE, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            trial.number, lr, momentum, weight_decay,
            augment, mosaic, mixup, hsv_h, hsv_s, hsv_v,
            map50
        ])

    return map50

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)  # adjust n_trials for more thorough tuning

print("Best mAP@0.5:", study.best_value)
print("Best hyperparameters:", study.best_params)


[I 2025-09-25 05:03:34,479] A new study created in memory with name: no-name-3749bd57-1ee2-4b68-98b9-7bc5b191a3ed
suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float

Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/continuous_fire-6/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.04314209026366733, hsv_s=0.5977817818881362, hsv_v=0.35958204344013295, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0031870177585876947, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.9746716512778438, mode=train, model=yolov8n.pt, momentum=0.6195743088843786, mosaic=0.8835560022102443, multi_scale=False, name=optuna_trial_0, nbs=64, nm

[I 2025-09-25 05:07:24,076] Trial 0 finished with value: 0.8384570424016744 and parameters: {'lr': 0.0031870177585876947, 'momentum': 0.6195743088843786, 'weight_decay': 2.3800319309704233e-05, 'augment': False, 'mosaic': 0.8835560022102443, 'mixup': 0.9746716512778438, 'hsv_h': 0.04314209026366733, 'hsv_s': 0.5977817818881362, 'hsv_v': 0.35958204344013295}. Best is trial 0 with value: 0.8384570424016744.


Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/continuous_fire-6/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.22903364441094848, hsv_s=0.6604140968788852, hsv_v=0.40303005674307046, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00023642745295315742, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.6492365761342122, mode=train, model=yolov8n.pt, momentum=0.6783020929087962, mosaic=0.09655897213202247, multi_scale=False, name=optuna_trial_1, nbs=64, n

[I 2025-09-25 05:11:16,615] Trial 1 finished with value: 0.8586367063643053 and parameters: {'lr': 0.00023642745295315742, 'momentum': 0.6783020929087962, 'weight_decay': 1.5204507468586434e-06, 'augment': True, 'mosaic': 0.09655897213202247, 'mixup': 0.6492365761342122, 'hsv_h': 0.22903364441094848, 'hsv_s': 0.6604140968788852, 'hsv_v': 0.40303005674307046}. Best is trial 1 with value: 0.8586367063643053.


...

Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/continuous_fire-6/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.17463215982382657, hsv_s=0.12834097804427866, hsv_v=0.03697511433334635, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=2.053893041491517e-05, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.39599176691774624, mode=train, model=yolov8n.pt, momentum=0.9749053535624814, mosaic=0.43864056699812193, multi_scale=False, name=optuna_trial_28, nbs=64,

[I 2025-09-25 07:00:18,478] Trial 28 finished with value: 0.8691895006117516 and parameters: {'lr': 2.053893041491517e-05, 'momentum': 0.9749053535624814, 'weight_decay': 4.727747604748384e-05, 'augment': True, 'mosaic': 0.43864056699812193, 'mixup': 0.39599176691774624, 'hsv_h': 0.17463215982382657, 'hsv_s': 0.12834097804427866, 'hsv_v': 0.03697511433334635}. Best is trial 18 with value: 0.9020871868219388.


Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/continuous_fire-6/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.05778732743100713, hsv_s=0.04304460773172705, hsv_v=0.08921299577433453, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=8.353213522332649e-05, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.9871304421482849, mode=train, model=yolov8n.pt, momentum=0.6020605705363736, mosaic=0.7224286206241203, multi_scale=False, name=optuna_trial_29, nbs=64, 

[I 2025-09-25 07:04:19,169] Trial 29 finished with value: 0.8664797693703928 and parameters: {'lr': 8.353213522332649e-05, 'momentum': 0.6020605705363736, 'weight_decay': 1.9774895046928887e-05, 'augment': False, 'mosaic': 0.7224286206241203, 'mixup': 0.9871304421482849, 'hsv_h': 0.05778732743100713, 'hsv_s': 0.04304460773172705, 'hsv_v': 0.08921299577433453}. Best is trial 18 with value: 0.9020871868219388.


Best mAP@0.5: 0.9020871868219388
Best hyperparameters: {'lr': 3.9910306660183916e-05, 'momentum': 0.7523582747603865, 'weight_decay': 8.010681158727771e-06, 'augment': True, 'mosaic': 0.5632826183751805, 'mixup': 0.16404941973095155, 'hsv_h': 0.0006803640146269552, 'hsv_s': 0.09680940833373346, 'hsv_v': 0.0775579831075962}


In [19]:
best_params = {
    'lr': 3.9910306660183916e-05,
    'momentum': 0.7523582747603865,
    'weight_decay': 8.010681158727771e-06,
    'augment': True,
    'mosaic': 0.5632826183751805,
    'mixup': 0.16404941973095155,
    'hsv_h': 0.0006803640146269552,
    'hsv_s': 0.09680940833373346,
    'hsv_v': 0.0775579831075962
}

DATA_YAML = "/content/continuous_fire-6/data.yaml"

final_model = YOLO("yolov8n.pt")

final_model.train(
    data=DATA_YAML,
    epochs=100,
    imgsz=640,
    batch=16,
    lr0=best_params['lr'],
    momentum=best_params['momentum'],
    weight_decay=best_params['weight_decay'],
    augment=best_params['augment'],
    mosaic=best_params['mosaic'],
    mixup=best_params['mixup'],
    hsv_h=best_params['hsv_h'],
    hsv_s=best_params['hsv_s'],
    hsv_v=best_params['hsv_v'],
    project=RESULTS_DIR,
    name='best_model',
    patience=10,
    exist_ok=True
)

Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/continuous_fire-6/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0006803640146269552, hsv_s=0.09680940833373346, hsv_v=0.0775579831075962, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=3.9910306660183916e-05, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.16404941973095155, mode=train, model=yolov8n.pt, momentum=0.7523582747603865, mosaic=0.5632826183751805, multi_scale=False, name=best_model, nbs=64, nm

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7cd597782240>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [20]:
from google.colab import files
files.download("/content/results/best_model/weights/best.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>